## DSPy program: events ranking

In [1]:
%load_ext autoreload
%load_ext dotenv
%dotenv -o

import os

# openai_api_key = os.getenv("OPENAI_API_KEY")
# print(f"OPENAI_API_KEY: {openai_api_key}")

In [2]:
%%capture
%pip install langchain langchain_community
%pip install faiss-gpu faiss-cpu
%pip install langchain-cohere
%pip install dspy-ai

In [1]:
import os

current_directory = os.getcwd()

workspace_directory = os.path.abspath(os.path.join(current_directory, ".."))
print(workspace_directory)

/home/izlobin/wd/events-planner-sst


In [4]:
import json

import pandas as pd

datasets_base_path = f"{workspace_directory}/task/datasets"
datasets_paths = [
    f"{datasets_base_path}/local-luma",
    f"{datasets_base_path}/local-meetup",
]

df_all = None
for dataset_path in datasets_paths:
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                data = json.load(f)
                df = pd.json_normalize(data)
                df["url"] = df["url"].apply(lambda x: x.split("?")[0])

                if "df_all" not in globals():
                    df_all = df
                else:
                    df_all = pd.concat([df_all, df], ignore_index=True)

df_preview = df_all.drop(columns=["description", "cover", "dateEnd"])
print(df_preview.head(30).to_markdown())

|    | url                                                                                                   | dateStart                | title                                                                                                        | hosts                                                                                       | group                                                   | address                                                | guests                                               |   attendees | tags                       | venue                     |   online | eventSeriesDates                                                                                                                                                                                                                                                                         |
|---:|:------------------------------------------------------------------------------------------------------|:-----------

In [5]:
url_to_labels = {
    "https://lu.ma/mw005iqm": {
        "title": "Pitch and Run 730am Thursday Early Bird",
        "non_commercial": 1.0,
        "non_commercial_explanation": "This event is not sponsored by any company, it is a community event",
        "popularity": 1.0,
        "popularity_explanation": "This event is very popular, ~250 people are attending",
        "free_admision": 1.0,
        "free_admision_explanation": "This event is completely free to attend",
        "no_additional_expenses": 1.0,
        "no_additional_expenses_explanation": "You don't have to pay for anything",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided",
        "venue_niceness": 0.0,
        "venue_niceness_explanation": "The venue is outdoors",
        "quietness": 0.5,
        "quietness_explanation": "The event is outdoors, so it's not very quiet",
        "uniqueness": 0.2,
        "uniqueness_explanation": "Running is not so unique activity",
        "proximity": 0.8,
        "proximity_explanation": "The event is in Chealsey, so transportation is easy",
    },
    "https://lu.ma/8vxdejas": {
        "title": "Pitch & Run - Prospect Park",
        "non_commercial": 1.0,
        "non_commercial_explanation": "This event is not sponsored by any company, it is a community event",
        "popularity": 1.0,
        "popularity_explanation": "This event is quite popular, ~200 people are attending",
        "free_admision": 1.0,
        "free_admision_explanation": "This event is completely free to attend",
        "no_additional_expenses": 1.0,
        "no_additional_expenses_explanation": "You don't have to pay for anything",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided",
        "venue_niceness": 0.0,
        "venue_niceness_explanation": "The venue is outdoors",
        "quietness": 0.5,
        "quietness_explanation": "The event is outdoors, so it's not very quiet",
        "uniqueness": 0.2,
        "uniqueness_explanation": "Running is not so unique activity",
        "proximity": 0.6,
        "proximity_explanation": "The event is in Brooklyn nearby Prospect Park a little out from Manhattan",
    },
    "https://lu.ma/founderunwindjan": {
        "title": "january founder's sauna club (hosted by andrew yeung x rho)",
        "non_commercial": 1.0,
        "non_commercial_explanation": "This event is not sponsored by any company, it is a community event",
        "popularity": 0.8,
        "popularity_explanation": "This event is quite popular, ~100 people are attending",
        "free_admision": 0.1,
        "free_admision_explanation": "The event is quite expensive to attend, about $60 for a class",
        "no_additional_expenses": 0.8,
        "no_additional_expenses_explanation": "You may want to buy drinks or food",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided",
        "venue_niceness": 1.0,
        "venue_niceness_explanation": "Sauna club is a nice venue",
        "quietness": 0.7,
        "quietness_explanation": "The event is indoors and not many attendees, so quite quiet",
        "uniqueness": 1.0,
        "uniqueness_explanation": "Sauna club is a unique activity",
        "proximity": 0.8,
        "proximity_explanation": "The event is in Brooklyn nearby Prospect Park a little out from Manhattan",
    },
    "https://lu.ma/li70qmdn": {
        "title": "January Finance Happy Hour",
        "non_commercial": 0.7,
        "non_commercial_explanation": "The host is self-promoting thier services",
        "popularity": 0.5,
        "popularity_explanation": "Attendance is unknown, assume moderate",
        "free_admision": 0.4,
        "free_admision_explanation": "The event is not expensive to attend, about $10 for a pass",
        "no_additional_expenses": 0.8,
        "no_additional_expenses_explanation": "You may want to buy drinks or food",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided, cause not mentioned",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided, cause not mentioned",
        "venue_niceness": 0.5,
        "venue_niceness_explanation": "Don't know the event venue, assume moderate",
        "quietness": 0.7,
        "quietness_explanation": "The event is indoors and not many attendees, so quite quiet",
        "uniqueness": 0.4,
        "uniqueness_explanation": "Not so unique, just a bar in West Village",
        "proximity": 0.6,
        "proximity_explanation": "The event is in Brooklyn nearby Prospect Park a little out from Manhattan",
    },
    "https://lu.ma/9mc2psu3": {
        "title": "NachoTuesday: SaaS & AI Founder + Investor Happy Hour",
        "non_commercial": 0.0,
        "non_commercial_explanation": "Very much commercial event, sponsored by very many companies",
        "popularity": 1.0,
        "popularity_explanation": "Attendance is big, over 350 people are attending, which is good",
        "free_admision": 0.8,
        "free_admision_explanation": "RSVP approval is required",
        "no_additional_expenses": 1.0,
        "no_additional_expenses_explanation": "Since catering is provided, you don't have to pay for anything",
        "drinks_provided": 0.8,
        "drinks_provided_explanation": "Sponsored drinks are provided by, though not an open bar",
        "food_provided": 0.6,
        "food_provided_explanation": "Appetizers are provided by the sponsors, but not full variety of meals is mentioned",
        "venue_niceness": 0.8,
        "venue_niceness_explanation": "Don't know the event venue, but lots of sponsors, so assume better than moderate",
        "quietness": 0.4,
        "quietness_explanation": "Big attendance, so not very quiet",
        "uniqueness": 0.8,
        "uniqueness_explanation": "Quite unique given the number of sponsors and communities",
        "proximity": 0.8,
        "proximity_explanation": "The event is in Manhattan, plus lots of sponsors, probably close to all transport",
    },
    "https://lu.ma/o0h2xzf4": {
        "title": "NYC B2B: Pre-Seed + Seed Founders & Investors Meetup",
        "non_commercial": 0.7,
        "non_commercial_explanation": "Sponsored and hosted by the same company, so not very commercial",
        "popularity": 0.8,
        "popularity_explanation": "Quite a big attendance, over 200 people are attending",
        "free_admision": 0.8,
        "free_admision_explanation": "RSVP approval is required",
        "no_additional_expenses": 0.8,
        "no_additional_expenses_explanation": "No minimum expenses are mentioned, but you may want to buy drinks or food",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided, cause not mentioned",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided, cause not mentioned",
        "venue_niceness": 0.5,
        "venue_niceness_explanation": "Not mentioned, assume moderate",
        "quietness": 0.4,
        "quietness_explanation": "Big attendance, so not very quiet",
        "uniqueness": 0.5,
        "uniqueness_explanation": "Don't know the event venue, assume moderate",
        "proximity": 0.5,
        "proximity_explanation": "Not mentioned, assume somewhere close to transport in Manhattan",
    },
    "https://lu.ma/0odxlgwd": {
        "title": "Making In America: Calling All Immigrant Founders & Creators | Visas, Skilled Immigration & Entrepreneurship",
        "non_commercial": 0.7,
        "non_commercial_explanation": "Sponsored and hosted by the same company, so not very commercial",
        "popularity": 0.9,
        "popularity_explanation": "Quite a big attendance, over 230 people are attending",
        "free_admision": 0.8,
        "free_admision_explanation": "RSVP approval is required",
        "no_additional_expenses": 1.0,
        "no_additional_expenses_explanation": "No minimum expenses are mentioned",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided, cause not mentioned",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided, cause not mentioned",
        "venue_niceness": 0.5,
        "venue_niceness_explanation": "Not mentioned, assume moderate",
        "quietness": 0.4,
        "quietness_explanation": "Big attendance, so not very quiet",
        "uniqueness": 0.5,
        "uniqueness_explanation": "Don't know the event venue, assume moderate",
        "proximity": 0.5,
        "proximity_explanation": "Not mentioned, assume somewhere close to transport in Manhattan",
    },
    "https://lu.ma/n7pmiw48": {
        "title": "Exceptional Yoga & Breathwork",
        "non_commercial": 0.7,
        "non_commercial_explanation": "Sponsored and hosted by the same company, so not very commercial",
        "popularity": 0.6,
        "popularity_explanation": "Moderate attendance about 25 people",
        "free_admision": 0.4,
        "free_admision_explanation": "RSVP approval is required, venue is limited so harder to get a pass",
        "no_additional_expenses": 0.9,
        "no_additional_expenses_explanation": "No minimum expenses are mentioned, but you might want to buy some water for yoga",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided, cause not mentioned",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided, cause not mentioned",
        "venue_niceness": 0.6,
        "venue_niceness_explanation": "Yoga venue should be nice, but not too much",
        "quietness": 0.8,
        "quietness_explanation": "Limited attendance, so should be very quiet",
        "uniqueness": 0.7,
        "uniqueness_explanation": "Yoga activity is quite unique",
        "proximity": 0.5,
        "proximity_explanation": "Not mentioned, assume somewhere close to transport in Manhattan",
    },
    "https://lu.ma/joo0gbrn": {
        "title": "Rethinking Hardware in the AI Era",
        "non_commercial": 0.4,
        "non_commercial_explanation": "A few sponsors are mentioned, so not very commercial",
        "popularity": 0.9,
        "popularity_explanation": "Very popular, ~350 people signed up",
        "free_admision": 0.7,
        "free_admision_explanation": "RSVP approval is required",
        "no_additional_expenses": 0.9,
        "no_additional_expenses_explanation": "No minimum expenses are mentioned",
        "drinks_provided": 0.0,
        "drinks_provided_explanation": "No drinks are provided, cause not mentioned",
        "food_provided": 0.0,
        "food_provided_explanation": "No food is provided, cause not mentioned",
        "venue_niceness": 0.5,
        "venue_niceness_explanation": "We don't know the venue, assume moderate",
        "quietness": 0.3,
        "quietness_explanation": "Lots of attendees, so not very quiet",
        "uniqueness": 0.7,
        "uniqueness_explanation": "Some C level suite (CEOs) speakers, so quite unique",
        "proximity": 0.5,
        "proximity_explanation": "Not mentioned, assume somewhere close to transport in Manhattan",
    },
}


def map_labels(row):
    labels = url_to_labels.get(row["url"], {})
    for key, value in labels.items():
        row[key] = value
    return row


df_labeled = df_all.apply(map_labels, axis=1)
df_labeled = df_labeled.dropna(subset=["non_commercial", "popularity", "free_admision", "drinks_provided", "venue_niceness", "quietness"])

df_preview = df_labeled.drop(columns=["description", "cover", "dateEnd"])
first_columns = ["url", "title", "dateStart", "non_commercial", "popularity", "free_admision", "drinks_provided", "venue_niceness", "quietness", "attendees"]
columns_order = first_columns + [col for col in df_preview.columns if col not in first_columns]
df_preview = df_preview[columns_order]
df_preview["title"] = df_preview["title"].apply(lambda x: x[:70] + "..." if len(x) > 70 else x)
print(df_preview.head(25).to_markdown())

|    | url                            | title                                                                     | dateStart                |   non_commercial |   popularity |   free_admision |   drinks_provided |   venue_niceness |   quietness |   attendees | address                                              | drinks_provided_explanation                              |   eventSeriesDates |   food_provided | food_provided_explanation                                                           | free_admision_explanation                                           | group                | guests                                     | hosts                                                                                       |   no_additional_expenses | no_additional_expenses_explanation                                               | non_commercial_explanation                                          |   online | popularity_explanation                                          |   proximit

In [6]:
import dspy
from typing import List
import json


def convert_to_dspy_labeled_examples(df: pd.DataFrame) -> List[dspy.Example]:
    examples = []
    for _, row in df.iterrows():
        example = dspy.Example(
            url=row["url"],
            dateStart=row["dateStart"],
            dateEnd=row["dateEnd"],
            title=row["title"],
            description=row["description"],
            hosts=row["hosts"],
            group=row["group"],
            address=row["address"],
            guests=row["guests"],
            attendees=row["attendees"],
            tags=row["tags"],
            venue=row["venue"],
            online=row["online"],
            eventSeriesDates=row["eventSeriesDates"],
            non_commercial=row["non_commercial"],
            popularity=row["popularity"],
            free_admision=row["free_admision"],
            no_additional_expenses=row["no_additional_expenses"],
            drinks_provided=row["drinks_provided"],
            food_provided=row["food_provided"],
            venue_niceness=row["venue_niceness"],
            quietness=row["quietness"],
            uniqueness=row["uniqueness"],
            proximity=row["proximity"],
        ).with_inputs("title", "description", "hosts", "group", "address", "guests", "attendees", "tags", "venue", "online", "eventSeriesDates")
        examples.append(example)
    return examples


def convert_to_dspy_examples(df: pd.DataFrame) -> List[dspy.Example]:
    examples = []
    for _, row in df.iterrows():
        example = dspy.Example(
            url=row["url"],
            dateStart=row["dateStart"],
            dateEnd=row["dateEnd"],
            title=row["title"],
            description=row["description"],
            hosts=row["hosts"],
            group=row["group"],
            address=row["address"],
            guests=row["guests"],
            attendees=row["attendees"],
            tags=row["tags"],
            venue=row["venue"],
            online=row["online"],
            eventSeriesDates=row["eventSeriesDates"],
        ).with_inputs("title", "description", "hosts", "group", "address", "guests", "attendees", "tags", "venue", "online", "eventSeriesDates")
        examples.append(example)
    return examples


labeled_examples = convert_to_dspy_labeled_examples(df_labeled)
all_examples = convert_to_dspy_examples(df_all)

print(f"total number of examples: {len(labeled_examples)}")
# single_line_json = json.dumps([example.toDict() for example in examples])
# print(single_line_json)

test_set = labeled_examples[0:5]
train_set = labeled_examples[5:10]
try_set = all_examples[20:25]

# print(f"testset size: {len(test_set)}")
# testset_single_line_json = json.dumps([example.toDict() for example in test_set])
# print(testset_single_line_json)
print(f"trainset size: {len(train_set)}")
trainset_single_line_json = json.dumps([example.toDict() for example in train_set])
print(trainset_single_line_json)

print(f"tryset size: {len(try_set)}")
tryset_single_line_json = json.dumps([example.toDict() for example in try_set])
print(tryset_single_line_json)
# multiline_json = json.dumps([example.toDict() for example in examples[0:1]], indent=4)
# print(multiline_json)

total number of examples: 9
trainset size: 4
[{"url": "https://lu.ma/o0h2xzf4", "dateStart": "2025-01-27T23:30:00.000Z", "dateEnd": "2025-01-28T01:30:00.000Z", "title": "NYC B2B: Pre-Seed + Seed Founders & Investors Meetup", "description": "<p>\u200b<a href=\"http://nycb2b.beehiiv.com\" target=\"_blank\" rel=\"nofollow noopener\">NYC B2B</a> is the premier community for B2B Founders, Operators, and Investors in NYC. <a href=\"http://nycb2b.beehiiv.com\" target=\"_blank\" rel=\"nofollow noopener\">NYC B2B</a> often hosts happy hour events for founders, builders, and investors based in NYC.</p><p>\u200b<!-- -->This is an exclusive event for B2B Pre-Seed and Seed stage founders and investors. </p><p>\u200b<!-- -->\u200b<strong>\u200bWant to sponsor this event? Email @ </strong><strong><a href=\"mailto:newyorkcityventures@gmail.com\" target=\"_blank\" rel=\"nofollow noopener\">newyorkcityventures@gmail.com</a></strong></p><p>\u200b<!-- -->\u200bThis meetup is hosted by Manan Modi. It\u2019

In [7]:
from typing import List
import dspy


class ScoringFeatures(dspy.Signature):
    """
    Extract scoring features from event data.
    """

    title: str = dspy.InputField(desc="event title")
    description: str = dspy.InputField(desc="event description")
    attendees: str = dspy.InputField(desc="the number of people attending the event")
    address: str = dspy.InputField(desc="event address")
    guests: list[str] = dspy.InputField(desc="event guests")

    non_commercial: float = dspy.OutputField(desc="non_commercial (0 - very commercial, 1 - not commercial at all)")
    popularity: float = dspy.OutputField(desc="popularity (0 - very unpopular, 1 - very popular)")
    free_admision: float = dspy.OutputField(desc="free_admision (0 - very expensive, 1 - completely free)")
    no_additional_expenses: float = dspy.OutputField(desc="no_additional_expenses (0 - lots of additional expenses, 1 - no additional expenses)")
    drinks_provided: float = dspy.OutputField(desc="drinks_provided (0 - no drinks provided, 1 - lots of drinks provided)")
    food_provided: float = dspy.OutputField(desc="food_provided (0 - no food provided, 1 - lots of food provided)")
    venue_niceness: float = dspy.OutputField(desc="venue_niceness (0 - very poor venue, 1 - very nice venue)")
    quietness: float = dspy.OutputField(desc="quietness (0 - very loud, 1 - very quiet)")
    uniqueness: float = dspy.OutputField(desc="uniqueness (0 - not unique, 1 - very unique)")
    proximity: float = dspy.OutputField(desc="proximity (0 - very far, 1 - very close)")


scoring_features_extractor = dspy.ChainOfThought(ScoringFeatures)


# class ScoringFeaturesAnnotatedInputs(dspy.Signature):
#     """
#     Extract scoring features from event data.
#     """

#     title: str = dspy.InputField(desc="event title")
#     description: str = dspy.InputField(desc="event description")
#     attendees: str = dspy.InputField(desc="the number of people attending the event")
#     address: str = dspy.InputField(desc="event address")
#     guests: list[str] = dspy.InputField(desc="event guests")

#     non_commercial: float = dspy.OutputField(desc="non_commercial (0 - very commercial, 1 - not commercial at all)")
#     non_commercial_explanation: str = dspy.InputField(desc="Explanation for non_commercial score")
#     popularity: float = dspy.OutputField(desc="popularity (0 - very unpopular, 1 - very popular)")
#     popularity_explanation: str = dspy.InputField(desc="Explanation for popularity score")
#     free_admision: float = dspy.OutputField(desc="free_admision (0 - very expensive, 1 - completely free)")
#     free_admision_explanation: str = dspy.InputField(desc="Explanation for free_admision score")
#     no_additional_expenses: float = dspy.OutputField(desc="no_additional_expenses (0 - lots of additional expenses, 1 - no additional expenses)")
#     no_additional_expenses_explanation: str = dspy.InputField(desc="Explanation for no_additional_expenses score")
#     drinks_provided: float = dspy.OutputField(desc="drinks_provided (0 - no drinks provided, 1 - lots of drinks provided)")
#     drinks_provided_explanation: str = dspy.InputField(desc="Explanation for drinks_provided score")
#     food_provided: float = dspy.OutputField(desc="food_provided (0 - no food provided, 1 - lots of food provided)")
#     food_provided_explanation: str = dspy.InputField(desc="Explanation for food_provided score")
#     venue_niceness: float = dspy.OutputField(desc="venue_niceness (0 - very poor venue, 1 - very nice venue)")
#     venue_niceness_explanation: str = dspy.InputField(desc="Explanation for venue_niceness score")
#     quietness: float = dspy.OutputField(desc="quietness (0 - very loud, 1 - very quiet)")
#     quietness_explanation: str = dspy.InputField(desc="Explanation for quietness score")
#     uniqueness: float = dspy.OutputField(desc="uniqueness (0 - not unique, 1 - very unique)")
#     uniqueness_explanation: str = dspy.InputField(desc="Explanation for uniqueness score")
#     proximity: float = dspy.OutputField(desc="proximity (0 - very far, 1 - very close)")
#     proximity_explanation: str = dspy.InputField(desc="Explanation for proximity score")


# scoring_features_annotated_inputs_extractor = dspy.ChainOfThought(ScoringFeaturesAnnotatedInputs)


# class ScoringFeaturesAnnotated(dspy.Signature):
#     """
#     Extract scoring features from event data.
#     """

#     title: str = dspy.InputField(desc="event title")
#     description: str = dspy.InputField(desc="event description")
#     attendees: str = dspy.InputField(desc="the number of people attending the event")
#     address: str = dspy.InputField(desc="event address")
#     guests: list[str] = dspy.InputField(desc="event guests")

#     non_commercial: float = dspy.OutputField(desc="non_commercial (0 - very commercial, 1 - not commercial at all)")
#     non_commercial_explanation: str = dspy.OutputField(desc="Explanation for non_commercial score")
#     popularity: float = dspy.OutputField(desc="popularity (0 - very unpopular, 1 - very popular)")
#     popularity_explanation: str = dspy.OutputField(desc="Explanation for popularity score")
#     free_admision: float = dspy.OutputField(desc="free_admision (0 - very expensive, 1 - completely free)")
#     free_admision_explanation: str = dspy.OutputField(desc="Explanation for free_admision score")
#     no_additional_expenses: float = dspy.OutputField(desc="no_additional_expenses (0 - lots of additional expenses, 1 - no additional expenses)")
#     no_additional_expenses_explanation: str = dspy.OutputField(desc="Explanation for no_additional_expenses score")
#     drinks_provided: float = dspy.OutputField(desc="drinks_provided (0 - no drinks provided, 1 - lots of drinks provided)")
#     drinks_provided_explanation: str = dspy.OutputField(desc="Explanation for drinks_provided score")
#     food_provided: float = dspy.OutputField(desc="food_provided (0 - no food provided, 1 - lots of food provided)")
#     food_provided_explanation: str = dspy.OutputField(desc="Explanation for food_provided score")
#     venue_niceness: float = dspy.OutputField(desc="venue_niceness (0 - very poor venue, 1 - very nice venue)")
#     venue_niceness_explanation: str = dspy.OutputField(desc="Explanation for venue_niceness score")
#     quietness: float = dspy.OutputField(desc="quietness (0 - very loud, 1 - very quiet)")
#     quietness_explanation: str = dspy.OutputField(desc="Explanation for quietness score")
#     uniqueness: float = dspy.OutputField(desc="uniqueness (0 - not unique, 1 - very unique)")
#     uniqueness_explanation: str = dspy.OutputField(desc="Explanation for uniqueness score")
#     proximity: float = dspy.OutputField(desc="proximity (0 - very far, 1 - very close)")
#     proximity_explanation: str = dspy.OutputField(desc="Explanation for proximity score")


# scoring_features_annotated_extractor = dspy.ChainOfThought(ScoringFeaturesAnnotated)

In [8]:
def scoring_features_metric(example: dspy.Example, prediction: dspy.Prediction, trace=None) -> float:
    """
    Computes a composite score for the scoring features.

    Args:
        example (dspy.Example): The dataset example containing expected scoring features.
        prediction (dspy.Prediction): The prediction from the DSPy scoring features extraction program.
        trace: Optional trace object for debugging.

    Returns:
        float: Composite score based on the equally weighted scoring features.
    """
    features = [
        "non_commercial",
        "popularity",
        "free_admision",
        "no_additional_expenses",
        "drinks_provided",
        "food_provided",
        "venue_niceness",
        "quietness",
        "uniqueness",
        "proximity",
    ]

    weights = {
        "non_commercial": 1.0,
        "popularity": 1.0,
        "free_admision": 1.0,
        "no_additional_expenses": 1.0,
        "drinks_provided": 1.0,
        "food_provided": 1.0,
        "venue_niceness": 1.0,
        "quietness": 1.0,
        "uniqueness": 1.0,
        "proximity": 1.0,
    }

    total_weight = sum(weights.values())
    score = 0.0

    for feature in features:
        expected_value = getattr(example, feature)
        predicted_value = getattr(prediction, feature)
        score += 1 - abs(expected_value - predicted_value)

    return score / total_weight


evaluate_correctness = dspy.Evaluate(devset=test_set, metric=scoring_features_metric, num_threads=24, display_progress=True, display_table=True)

In [9]:
lm = dspy.LM(model="openai/gpt-4o-mini")
dspy.settings.configure(lm=lm)

evaluate_correctness(scoring_features_extractor, devset=test_set)

Average Metric: 3.82 / 5 (76.4%): 100%|██████████| 5/5 [00:00<00:00, 191.35it/s]

2025/01/30 14:28:50 INFO dspy.evaluate.evaluate: Average Metric: 3.8200000000000003 / 5 (76.4%)


,url,dateStart,dateEnd,title,description,hosts,group,address,guests,attendees,...,pred_popularity,pred_free_admision,pred_no_additional_expenses,pred_drinks_provided,pred_food_provided,pred_venue_niceness,pred_quietness,pred_uniqueness,pred_proximity,scoring_features_metric
0,https://lu.ma/mw005iqm,2025-01-23T12:30:00.000Z,2025-01-23T13:30:00.000Z,Pitch and Run 730am Thursday Early Bird,<p>​<!-- -->Meet at 730am </p><p>​<!-- -->Intros till 740am</p><p>...,"[Kevin Weatherman, Chaitenya Razdan]",Pitch and Run,"Le Pain Quotidien, New York, New York","[Lus Kairos, Josh]",258,...,0.9,1.0,0.9,0.0,0.0,0.8,0.7,0.6,0.9,✔️ [0.750]
1,https://lu.ma/8vxdejas,2025-01-28T12:30:00.000Z,2025-01-28T13:30:00.000Z,Pitch & Run - Prospect Park,<p>​<!-- -->Meet at 7:30AM at Grand Army Plaza</p><p>​<!-- -->Rout...,"[Kevin Weatherman, Shahn Andersen, Jay Sehgal, Charlie O'Donnell, ...",Pitch and Run,"Grand Army Plaza Greenmarket, Kings County, New York","[Rafaela Gadelha, AI Founders Connect]",197,...,0.8,1.0,0.9,0.2,0.1,0.8,0.6,0.7,1.0,✔️ [0.750]
2,https://lu.ma/founderunwindjan,2025-01-27T02:00:00.000Z,2025-01-27T03:30:00.000Z,january founder's sauna club (hosted by andrew yeung x rho),<p>​<strong>you are invited to</strong><strong><em> the january fo...,"[Andrew Yeung, Justin Wolz, Othership]",Andrew's Tech Events,"Othership Flatiron, New York, New York","[Jaiya Gill, Ami Yoshimura]",104,...,0.7,0.2,0.6,0.3,0.1,0.9,0.8,0.9,0.9,✔️ [0.860]
3,https://lu.ma/li70qmdn,2025-01-23T23:30:00.000Z,2025-01-24T01:30:00.000Z,January Finance Happy Hour,<p>​<!-- -->Join us for the 11th monthly finance happy hour! Wheth...,[Fiona Tran],,"New York, New York","[Niko LeMieux, Christopher Truong]",86,...,0.8,0.9,0.7,0.8,0.5,0.7,0.4,0.6,0.9,✔️ [0.630]
4,https://lu.ma/9mc2psu3,2025-01-28T22:00:00.000Z,2025-01-29T01:00:00.000Z,NachoTuesday: SaaS & AI Founder + Investor Happy Hour,"<p>​<!-- -->NachoNacho - The B2B SaaS + Services Marketplace, Fide...","[Andy Karuza, Charlie Stephens, Nicole Haverick, Sanjay Goel, Chri...",,"New York, New York","[Andrea, Laurent]",322,...,0.9,0.8,0.7,1.0,1.0,0.9,0.6,0.8,1.0,✔️ [0.830]


76.4

In [10]:
mipro_optimizer = dspy.MIPROv2(
    metric=scoring_features_metric,
    auto="medium",
)

optimized_scoring_features_extractor = mipro_optimizer.compile(
    scoring_features_extractor,
    trainset=train_set,
    max_bootstrapped_demos=4,
    requires_permission_to_run=False,
    minibatch=False,
    seed=1,
)

2025/01/30 14:28:54 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 3

2025/01/30 14:28:54 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/30 14:28:54 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/30 14:28:54 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


100%|██████████| 1/1 [00:05<00:00,  5.26s/it]


Bootstrapped 1 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.
Bootstrapping set 4/19


100%|██████████| 1/1 [00:00<00:00, 496.43it/s]


...

2025/01/30 14:30:20 INFO dspy.evaluate.evaluate: Average Metric: 2.2199999999999998 / 3 (74.0%)
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 74.0 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 16'].
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [73.67, 72.67, 72.67, 72.33, 72.67, 70.33, 73.0, 72.0, 72.33, 72.67, 72.67, 73.0, 73.0, 73.0, 74.0, 74.0, 72.33, 74.0, 74.0, 72.67, 74.0, 73.0, 74.0, 74.0]
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 74.0
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 24 / 25 =====



Average Metric: 2.22 / 3 (74.0%): 100%|██████████| 3/3 [00:00<00:00, 2033.44it/s]

2025/01/30 14:30:20 INFO dspy.evaluate.evaluate: Average Metric: 2.2199999999999998 / 3 (74.0%)
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 74.0 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 15'].
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [73.67, 72.67, 72.67, 72.33, 72.67, 70.33, 73.0, 72.0, 72.33, 72.67, 72.67, 73.0, 73.0, 73.0, 74.0, 74.0, 72.33, 74.0, 74.0, 72.67, 74.0, 73.0, 74.0, 74.0, 74.0]
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 74.0
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====



Average Metric: 2.18 / 3 (72.7%): 100%|██████████| 3/3 [00:00<00:00, 2118.69it/s]

2025/01/30 14:30:20 INFO dspy.evaluate.evaluate: Average Metric: 2.1799999999999997 / 3 (72.7%)
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 72.67 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 15'].
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [73.67, 72.67, 72.67, 72.33, 72.67, 70.33, 73.0, 72.0, 72.33, 72.67, 72.67, 73.0, 73.0, 73.0, 74.0, 74.0, 72.33, 74.0, 74.0, 72.67, 74.0, 73.0, 74.0, 74.0, 74.0, 72.67]
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 74.0
2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/30 14:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 74.0!


In [15]:
cost = sum([x["cost"] for x in lm.history if x["cost"] is not None])
print(cost)

dspy.inspect_history(n=1)

0.00246555




[2025-01-30T14:31:54.312365]

System message:

Your input fields are:
1. `title` (str): event title
2. `description` (str): event description
3. `attendees` (str): the number of people attending the event
4. `address` (str): event address
5. `guests` (list[str]): event guests

Your output fields are:
1. `rationale` (str): ${produce the proximity}. We ...
2. `non_commercial` (float): non_commercial (0 - very commercial, 1 - not commercial at all)
3. `popularity` (float): popularity (0 - very unpopular, 1 - very popular)
4. `free_admision` (float): free_admision (0 - very expensive, 1 - completely free)
5. `no_additional_expenses` (float): no_additional_expenses (0 - lots of additional expenses, 1 - no additional expenses)
6. `drinks_provided` (float): drinks_provided (0 - no drinks provided, 1 - lots of drinks provided)
7. `food_provided` (float): food_provided (0 - no food provided, 1 - lots of food provided)
8. `venue_niceness` (float): venue_niceness (0 - very poor ven

In [12]:
# third run (with explanation as outputs)

lm = dspy.LM(model="openai/gpt-4o-mini")
dspy.settings.configure(lm=lm)

evaluate_correctness(scoring_features_extractor, devset=test_set)

Average Metric: 3.82 / 5 (76.4%): 100%|██████████| 5/5 [00:00<00:00, 2080.10it/s]

2025/01/30 14:31:20 INFO dspy.evaluate.evaluate: Average Metric: 3.8200000000000003 / 5 (76.4%)


,url,dateStart,dateEnd,title,description,hosts,group,address,guests,attendees,...,pred_popularity,pred_free_admision,pred_no_additional_expenses,pred_drinks_provided,pred_food_provided,pred_venue_niceness,pred_quietness,pred_uniqueness,pred_proximity,scoring_features_metric
0,https://lu.ma/mw005iqm,2025-01-23T12:30:00.000Z,2025-01-23T13:30:00.000Z,Pitch and Run 730am Thursday Early Bird,<p>​<!-- -->Meet at 730am </p><p>​<!-- -->Intros till 740am</p><p>...,"[Kevin Weatherman, Chaitenya Razdan]",Pitch and Run,"Le Pain Quotidien, New York, New York","[Lus Kairos, Josh]",258,...,0.9,1.0,0.9,0.0,0.0,0.8,0.7,0.6,0.9,✔️ [0.750]
1,https://lu.ma/8vxdejas,2025-01-28T12:30:00.000Z,2025-01-28T13:30:00.000Z,Pitch & Run - Prospect Park,<p>​<!-- -->Meet at 7:30AM at Grand Army Plaza</p><p>​<!-- -->Rout...,"[Kevin Weatherman, Shahn Andersen, Jay Sehgal, Charlie O'Donnell, ...",Pitch and Run,"Grand Army Plaza Greenmarket, Kings County, New York","[Rafaela Gadelha, AI Founders Connect]",197,...,0.8,1.0,0.9,0.2,0.1,0.8,0.6,0.7,1.0,✔️ [0.750]
2,https://lu.ma/founderunwindjan,2025-01-27T02:00:00.000Z,2025-01-27T03:30:00.000Z,january founder's sauna club (hosted by andrew yeung x rho),<p>​<strong>you are invited to</strong><strong><em> the january fo...,"[Andrew Yeung, Justin Wolz, Othership]",Andrew's Tech Events,"Othership Flatiron, New York, New York","[Jaiya Gill, Ami Yoshimura]",104,...,0.7,0.2,0.6,0.3,0.1,0.9,0.8,0.9,0.9,✔️ [0.860]
3,https://lu.ma/li70qmdn,2025-01-23T23:30:00.000Z,2025-01-24T01:30:00.000Z,January Finance Happy Hour,<p>​<!-- -->Join us for the 11th monthly finance happy hour! Wheth...,[Fiona Tran],,"New York, New York","[Niko LeMieux, Christopher Truong]",86,...,0.8,0.9,0.7,0.8,0.5,0.7,0.4,0.6,0.9,✔️ [0.630]
4,https://lu.ma/9mc2psu3,2025-01-28T22:00:00.000Z,2025-01-29T01:00:00.000Z,NachoTuesday: SaaS & AI Founder + Investor Happy Hour,"<p>​<!-- -->NachoNacho - The B2B SaaS + Services Marketplace, Fide...","[Andy Karuza, Charlie Stephens, Nicole Haverick, Sanjay Goel, Chri...",,"New York, New York","[Andrea, Laurent]",322,...,0.9,0.8,0.7,1.0,1.0,0.9,0.6,0.8,1.0,✔️ [0.830]


76.4

In [14]:
for example in try_set[:10]:
    result = optimized_scoring_features_extractor(
        url=example.url,
        dateStart=example.dateStart,
        dateEnd=example.dateEnd,
        title=example.title,
        description=example.description,
        hosts=example.hosts,
        group=example.group,
        address=example.address,
        guests=example.guests,
        attendees=example.attendees,
        tags=example.tags,
        venue=example.venue,
        online=example.online,
        eventSeriesDates=example.eventSeriesDates,
    )
    print(f"url: {example.url}, title: {example.title}")

    result_dict = result.toDict()
    # reasoning = result_dict.pop("reasoning")
    # result_dict["reasoning"] = reasoning
    result_json = json.dumps(result_dict)
    print(result_json)

url: https://www.meetup.com/shutupandwritenyc/events/305409078/, title: Shut Up & Write!® [Brooklyn, NY/Brooklyn Heights Library] 
{"rationale": "The event promotes a supportive writing environment, encouraging participants to focus on their writing in a communal setting. The proximity is high as it is located in Brooklyn, a well-connected area, making it easily accessible for local attendees.", "non_commercial": 0.9, "popularity": 0.4, "free_admision": 1.0, "no_additional_expenses": 0.8, "drinks_provided": 0.0, "food_provided": 0.0, "venue_niceness": 0.7, "quietness": 0.8, "uniqueness": 0.6, "proximity": 0.9}
url: https://www.meetup.com/lets-go-social-club-nyc/events/305008405/, title: Let's Go to a Cocktail Party! 
{"rationale": "The event is a social cocktail party aimed at a community of individuals aged 40 and above, providing a relaxed atmosphere for mingling. The venue is a rooftop at a hotel in New York, which enhances its appeal. Proximity is high due to the central location i

In [16]:
optimized_scoring_features_extractor.save("optimized_extractor.json")

In [ ]:
loaded_scoring_features_extractor = dspy.ChainOfThought(ScoringFeatures)
loaded_scoring_features_extractor.load("optimized_extractor.json")